In [ ]:
# FIAP - Curso IA para Devs
# Tech Challenge 03 
# Problema: 
#  -- No Tech Challenge desta fase, você precisa executar o fine-tuning de um 
#  -- foundation model (Llama, BERT, MISTRAL etc.), utilizando o dataset "The
#  -- AmazonTitles-1.3MM". O modelo treinado deverá:
#
# Grupo 44
# Francisco Antonio Guilherme
# fagn2013@gmail.com

# Marcelo Lima Gomes
# marcelolimagomes@gmail.com

# FELIPE MORAES DOS SANTOS
# felipe.moraes.santos2014@gmail.com

In [1]:
# Ambiente configurado para treinamento local em um PC com Placa de Vídeo Nvidia RTX-3060 12GB

# Utilizando miniconda, instalado em um Linux Ubuntu conforme orientações do link: https://docs.anaconda.com/miniconda/
# Utilizando miniconda para criação do ambiente do unsloth conforme orientação no link: https://docs.unsloth.ai/get-started/installation/conda-install

# >> Para configurar o ambiente, remova o comentário ("#") das linhas abaixo e execute os comandos.
# Lembre-se de instalar o miniconda previamente.

#!pip install nbformat
#!conda install -c conda-forge ipywidgets
#!conda create --name unsloth_env python=3.10 pytorch-cuda=12.1 pytorch cudatoolkit xformers -c pytorch -c nvidia -c xformers -y
#!conda activate unsloth_env
#!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
#!pip install --no-deps "trl<0.9.0" peft accelerate bitsandbytes

In [1]:
# Esse notebook tem como objetivo realizar o treinamento do modelo utilizando do dataset tratado anteriormente

import helper # Biblioteca local para apoio ao desenvolvimento
import torch  # Biblioteca fundamental para deep learning, usada para criar e treinar modelos de redes neurais. 
import datasets  # Biblioteca para carregar e preparar conjuntos de dados de diferentes formatos para treinamento de modelos.
from trl import SFTTrainer  # Classe para treinar modelos de linguagem de forma supervisionada, ajustando um modelo pré-treinado a uma tarefa específica.
from transformers import TrainingArguments  # Define os argumentos de treinamento, como número de épocas, tamanho do lote, etc.
from unsloth import is_bfloat16_supported  # Função para verificar se a GPU suporta o formato de ponto flutuante bfloat16, que pode acelerar o treinamento.
from unsloth import FastLanguageModel  # Classe para criar modelos de linguagem otimizados para maior velocidade e eficiência.

print("Versão PyTorch:", torch.__version__)
print("Versão CuDa:", torch.version.cuda)
print("Suporta Precisão Float 16 bits:", is_bfloat16_supported())

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Versão PyTorch: 2.4.1
Versão CuDa: 12.1
Suporta Precisão Float 16 bits True


In [3]:
max_seq_length = 2048  # Escolha qualquer um! Nós damos suporte automático ao RoPE Scaling internamente!
dtype = None  # None para detecção automática. Float16 para Tesla T4, V100, Bfloat16 para Ampere+
load_in_4bit = False  # Use quantização de 4 bits para reduzir o uso de memória. Pode ser Falso.

In [4]:
model_name = 'tinyllama_finetuned' # Nome do modelo gravado localmente.
raw_model, tokenizer = helper.get_model_by_name(model_name, max_seq_length, dtype, load_in_4bit)  # Carrega modelo na memória

Model Name: tinyllama_finetuned
==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 3060. Max memory: 11.65 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.9 patched 22 layers with 22 QKV layers, 22 O layers and 22 MLP layers.


In [5]:
# Prompt no padrão Alpaca é, em essência, uma forma estruturada de instrução que você fornece a um modelo de linguagem 
# para direcionar sua resposta. Essa estrutura é projetada para maximizar a qualidade e a relevância das respostas geradas pelo modelo, 
# tornando as interações mais naturais e informativas.
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Write a book review.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Marcador que indica o fim de uma sequência de tokens

#   Função para formatar o dataset em um conjunto de prompts para serem imputados na sequência de fine tuning do modelo
#   examples: Dataset contendo dados para serem treinados
def formatting_prompts_func(examples):
    inputs       = examples["title"]
    outputs      = examples["content"]
    texts = []
    for input, output in zip(inputs, outputs):
        # É necessário adicionar EOS_TOKEN, caso contrário sua geração continuará para sempre!
        text = alpaca_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

# Carrega um dataset préviamente processado, onde foi aplicado a limpeza dos dados.
# Esse processo é executado no notebook "limpeza_dataset.ipynb"
dataset = datasets.Dataset.from_csv('../data/trn_sample.csv', sep=';')
dataset = dataset.map(formatting_prompts_func, batched=True,)
dataset

Dataset({
    features: ['title', 'content', 'text'],
    num_rows: 10000
})

In [6]:
# Define e carrega as preferências para treinamento do modelo informado
model = FastLanguageModel.get_peft_model(
    # Contem o modelo carregado préviamente em memória
    raw_model,
    # Classificação da decomposição de baixa classificação para fatoração de matrizes de peso. Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    r=16,  
    # Seleciona os módulos para fazer o ajuste fino. Você pode remover alguns para reduzir o uso de memória e tornar o treinamento 
    # mais rápido, mas não sugerimos isso. Apenas treine em todos os módulos!
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    # Fator de escala para a contribuição das matrizes de baixa classificação.
    lora_alpha=16,
    # Probabilidade de zerar elementos em matrizes de baixa classificação para regularização.
    lora_dropout=0,  # Suporta any, mas = 0 é otimizado
    # Deixe como 0 para um treino mais rápido e com menos over-fit!
    bias="none",    # Suporta any, mas = "none" é otimizado
    # [NEW] "unsloth" usa 30% menos VRAM e se adapta a tamanhos de lote 2x maiores!
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    # Ativa o Rank-Stabilized LoRA (RSLora).
    use_rslora=True,  
    # Configuração para LoftQ, um método de quantização para os pesos do backbone e inicialização de camadas LoRA.
    loftq_config=None,  
  )

Unsloth: Already have LoRA adapters! We shall skip this step.


In [8]:
# Nome do modelo a ser salvo
save_name = 'tinyllama_finetuned_2'

# Definição do Trainer para o modelo supervisionado SFT (Supervised Fine-Tuning)
trainer = SFTTrainer(
    # O modelo que será treinado
    model=model,
    # O tokenizador que será utilizado para preparar o texto para o modelo
    tokenizer=tokenizer,
    # O dataset que será utilizado no processo de treinamento
    train_dataset=dataset,
    # O campo do dataset que contém o texto de entrada
    dataset_text_field="text",
    # Comprimento máximo das sequências de entrada
    max_seq_length=max_seq_length,
    # Número de processos paralelos para pré-processar os dados
    dataset_num_proc=2,
    # O packing pode ser usado para acelerar o treinamento em até 5x para sequências curtas, porém está desativado
    packing=False,
    # Argumentos relacionados ao treinamento, configurados usando a classe `TrainingArguments`
    args=TrainingArguments(
        # Tamanho do lote (batch) de treinamento por dispositivo (GPU ou CPU)
        per_device_train_batch_size=128,
        # Número de passos de acumulação de gradiente antes de atualizar os pesos
        gradient_accumulation_steps=1,
        # Quantidade de passos de aquecimento antes de iniciar o treinamento efetivo
        warmup_steps=5,
        # Número de épocas (passadas completas pelo dataset) que o modelo será treinado
        num_train_epochs=30,
        # Caminho onde os resultados e checkpoints do modelo serão salvos
        output_dir=save_name,
        # Estratégia de salvamento dos checkpoints durante o treinamento, definida para salvar a cada N passos
        save_strategy="steps",
        # Número de passos entre cada salvamento do modelo
        save_steps=10,
        # Taxa de aprendizado inicial utilizada no otimizador
        learning_rate=2e-4,
        # Ativa cálculos em precisão mista com FP16 se o hardware suportar, para melhorar a performance
        fp16=not is_bfloat16_supported(),
        # Usa precisão mista com bf16 (formato bfloat16) se o hardware suportar, geralmente para GPUs mais recentes
        bf16=is_bfloat16_supported(),
        # Número de passos entre cada registro de logs do treinamento
        logging_steps=1,
        # Define o otimizador AdamW (Adam com decaimento de peso) com 8 bits, mais eficiente em memória
        optim="adamw_8bit",
        # Taxa de decaimento de peso para regularização do modelo
        weight_decay=0.01,
        # Tipo de scheduler de taxa de aprendizado, definido como linear
        lr_scheduler_type="linear",
        # Semente aleatória para garantir reprodutibilidade do treinamento
        seed=3407,
    ),
)

In [9]:
# Imprime o dispositivo utilizado para treinamento e a quantidade de memória RAM disponível
start_gpu_memory, max_memory = helper.print_start_memory_usage()

GPU = NVIDIA GeForce RTX 3060. Max memory = 11.65 GB.
2.16 GB of memory reserved.


In [10]:
# Inicia o treinamento do modelo com o trainer configurado e armazena as estatísticas do treinamento na variável `trainer_stats`

# trainer.train() é o método que executa o processo de treinamento. Ele utiliza os parâmetros definidos no SFTTrainer, como o dataset, o modelo, e os argumentos de treinamento.
# O método retorna um objeto com as estatísticas do treinamento, que podem incluir informações como a perda (loss), tempo de treinamento, e outras métricas relevantes.
# As estatísticas do treinamento são armazenadas na variável `trainer_stats` para posterior análise ou salvamento.
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 10,000 | Num Epochs = 30
O^O/ \_/ \    Batch size per device = 128 | Gradient Accumulation steps = 1
\        /    Total batch size = 128 | Total steps = 2,370
 "-____-"     Number of trainable parameters = 12,615,680


  0%|          | 0/2370 [00:00<?, ?it/s]

{'loss': 0.5612, 'grad_norm': 0.7308847904205322, 'learning_rate': 4e-05, 'epoch': 0.01}
{'loss': 0.6093, 'grad_norm': 0.7540752291679382, 'learning_rate': 8e-05, 'epoch': 0.03}
{'loss': 0.5514, 'grad_norm': 0.8564906716346741, 'learning_rate': 0.00012, 'epoch': 0.04}
{'loss': 0.5589, 'grad_norm': 0.9263126254081726, 'learning_rate': 0.00016, 'epoch': 0.05}
{'loss': 0.5663, 'grad_norm': 0.7809894680976868, 'learning_rate': 0.0002, 'epoch': 0.06}
{'loss': 0.6248, 'grad_norm': 0.8446977138519287, 'learning_rate': 0.0001999154334038055, 'epoch': 0.08}
{'loss': 0.5466, 'grad_norm': 1.0597440004348755, 'learning_rate': 0.000199830866807611, 'epoch': 0.09}
{'loss': 0.607, 'grad_norm': 1.1898504495620728, 'learning_rate': 0.00019974630021141648, 'epoch': 0.1}
{'loss': 0.6152, 'grad_norm': 0.8875634074211121, 'learning_rate': 0.000199661733615222, 'epoch': 0.11}
{'loss': 0.5704, 'grad_norm': 0.973828136920929, 'learning_rate': 0.00019957716701902748, 'epoch': 0.13}
{'loss': 0.6735, 'grad_norm'

In [11]:
# Imprime o dispositivo utilizado para treinamento e a quantidade de memória RAM consumida no processo de treinamento
helper.print_final_memory_usage(start_gpu_memory, max_memory, trainer_stats)

23663.7741 seconds used for training.
394.4 minutes used for training.
Peak reserved memory = 9.748 GB.
Peak reserved memory for training = 7.588 GB.
Peak reserved memory % of max memory = 83.674 %.
Peak reserved memory for training % of max memory = 65.133 %.


In [12]:
# Salva o modelo treinado localmente no disco o que permite posterior consumo para realização das inferências e geração de texto
model.save_pretrained(save_name)  # Local saving
tokenizer.save_pretrained(save_name)

('tinyllama_finetuned_2/tokenizer_config.json',
 'tinyllama_finetuned_2/special_tokens_map.json',
 'tinyllama_finetuned_2/tokenizer.model',
 'tinyllama_finetuned_2/added_tokens.json',
 'tinyllama_finetuned_2/tokenizer.json')

In [13]:
# Salva o modelo no padrão GUFF treinado localmente no disco o que permite posterior consumo para realização das inferências e geração de texto
# O formato GGUF (um formato de checkpoint) com quantização em f16 (precisão de 16 bits)
model.save_pretrained_gguf(save_name + "f16_2", tokenizer, quantization_method='f16')  # Local saving

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 12.67 out of 31.17 RAM for saving.


100%|██████████| 22/22 [00:00<00:00, 104.06it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Unsloth: Converting llama model. Can use fast conversion = True.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['f16'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at tinyllama_finetuned_2f16_2 into f16 GGUF format.
The output location will be ./tinyllama_finetuned_2f16_2/unsloth.F16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: tinyllama_finetuned_2f16_2
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model part 'model.safetensors'
INFO:hf-to-gguf:output.weight,               torch.bfloat16 --> F16, shape = {2048, 32000}
INFO:hf-to-gguf:token_embd.weight,           torch.bfloat16 --> F16, shape = {2048, 32000}
INFO:hf-